In [2]:
import os
os.chdir("../")

In [3]:
os.environ['MLFLOW_TRACKING_URI']="https://dagshub.com/avimittal30/End-to-End-ML.mlflow" 
os.environ['MLFLOW_TRACKING_USERNAME']="avimittal30"
os.environ['MLFLOW_TRACKING_PASSWORD']="d2de36d373645cbd9bf6e2ff0367c8d679b74c41"

In [27]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir:Path
    test_data_path:Path
    model_path:Path
    all_params:dict
    metric_file_name:Path
    target_column:str
    mlflow_uri:str

In [30]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self)->ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.GradientBoostingClassifier
        schema=self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/avimittal30/End-to-End-ML.mlflow"
        )
        return model_evaluation_config

In [31]:
from sklearn.metrics import classification_report, recall_score, precision_score, accuracy_score
import os
import pandas as pd
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [45]:
class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config=config
        
    def eval_metrics(self, actual, pred):
        acc=accuracy_score(actual,pred)
        precision=precision_score(actual,pred)    
        recall=recall_score(actual,pred)
        return acc, precision,recall
    
    def log_into_mlflow(self):
        test_data=pd.read_csv(self.config.test_data_path)
        model=joblib.load(self.config.model_path)

        test_x=test_data.drop([self.config.target_column], axis=1)
        test_y=test_data[[self.config.target_column]]
        
        tracking_url_type_store=urlparse(mlflow.get_artifact_uri()).scheme
        mlflow.set_registry_uri(self.config.mlflow_uri)
        
        if mlflow.active_run():
            mlflow.end_run()

        with mlflow.start_run():
            predicted_class=model.predict(test_x)
            (acc, precision, recall)=self.eval_metrics(test_y,predicted_class)
            
            # Saving metrics as local
            scores={"acc":acc, "precision":precision, "recall":recall}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"precision": precision, "recall": recall, "accuracy": acc })

            if tracking_url_type_store!="file":
                mlflow.sklearn.log_model(model, "mymodel", registered_model_name="XGBClassifier")
            else:
                mlflow.sklearn.log_model(model, "mymodel")

In [46]:
config=ConfigurationManager()
model_evaluation_config=config.get_model_evaluation_config()
model_evaluation_config=ModelEvaluation(config=model_evaluation_config)
model_evaluation_config.log_into_mlflow()

[2024-03-11 02:25:44,460:INFO:common: yaml file: config\config.yaml loaded successfully]
[2024-03-11 02:25:44,467:INFO:common: yaml file: params.yaml loaded successfully]
[2024-03-11 02:25:44,467:INFO:common: yaml file: schema.yaml loaded successfully]
[2024-03-11 02:25:44,467:INFO:common: created directory at: artifacts]


FileNotFoundError: [Errno 2] No such file or directory: 'artifacts\\model_evaluation\\metrics.json'

In [36]:
mlflow.end_run()